# NextGEOSS
## Invoke the Territorial planning classification service using JupyterLab

### Inspect the OGC WPS capabilities

In [20]:
from owslib.wps import WebProcessingService
from owslib.wps import monitorExecution
from owslib.wps import printInputOutput

In [21]:
wps_url = "http://tp-classification.deimos.nextgeoss.terradue.com/wps/WebProcessingService"

In [22]:
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

In [23]:
wps.getcapabilities()

In [24]:
for operation in wps.operations:
    print (operation.name)

GetCapabilities
DescribeProcess
Execute


In [25]:
for process in wps.processes:
    print (process.identifier)

com.terradue.wps_oozie.process.OozieAbstractAlgorithm


### Inspect the OGC WPS process information and inputs

In [12]:
process_id = 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm'
process = wps.describeprocess(process_id)



In [13]:
process.title

'NextGEOSS7.1 Territorial Classification'

In [14]:
process.abstract

'Workflow of the territorial planning pilot forTerritorial Classification'

In [15]:
for input in process.dataInputs:
    printInputOutput(input)
    print ('***')

 identifier=enclosure_urls, title=Enclosure URI, abstract=List of data enclosure URI, data type=string
 Any value allowed
 Default Value: aHR0cHM6Ly9zY2lodWIuY29wZXJuaWN1cy5ldS9kaHVzL29kYXRhL3YxL1Byb2R1Y3RzKCc2MjVmNWQwYy05YmIyLTQ5YmEtODg0MS1lYmE3ZTFiNTJlZGInKS8kdmFsdWU=,aHR0cHM6Ly9zY2lodWIuY29wZXJuaWN1cy5ldS9kaHVzL29kYXRhL3YxL1Byb2R1Y3RzKCcyZGYwMmRlZS0yMjk0LTRjNTQtYmIwZi05M2ZhMjU0MTUyMDInKS8kdmFsdWU= 
 minOccurs=1, maxOccurs=100
***


### Prepare the execution request

In [16]:
inputs = []
inputs.append(('enclosure_urls','aHR0cHM6Ly9zY2lodWIuY29wZXJuaWN1cy5ldS9kaHVzL29kYXRhL3YxL1Byb2R1Y3RzKCc2MjVmNWQwYy05YmIyLTQ5YmEtODg0MS1lYmE3ZTFiNTJlZGInKS8kdmFsdWU=,aHR0cHM6Ly9zY2lodWIuY29wZXJuaWN1cy5ldS9kaHVzL29kYXRhL3YxL1Byb2R1Y3RzKCcyZGYwMmRlZS0yMjk0LTRjNTQtYmIwZi05M2ZhMjU0MTUyMDInKS8kdmFsdWU='))

outputs = [('result_osd',True)]

### Execute and monitor the request

In [17]:
execution = wps.execute(process_id, inputs, outputs)

In [18]:
monitorExecution(execution)

In [19]:
execution.isSucceded()

True

### Access the results' OpenSearch Description (osd)

In [19]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [20]:
results_osd = execution.processOutputs[0].reference

In [22]:
print (results_osd)

http://tp-classification.deimos.nextgeoss.terradue.com/wps/RetrieveResultServlet?id=7aea2823-52fe-4594-911a-714fed79fa8bresult_osd.4e96476e-6488-4e45-9c1f-45fe48eef73f


In [23]:
import requests
r = requests.get(results_osd)
r.text

'<?xml version=\'1.0\' encoding=\'UTF-8\'?><ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" href="http://tp-classification.deimos.nextgeoss.terradue.com/sbws/wps/workflow/0000006-200401000007612-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>'

In [24]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
root.attrib['href']

'http://tp-classification.deimos.nextgeoss.terradue.com/sbws/wps/workflow/0000006-200401000007612-oozie-oozi-W/results/description'

### Perform an OpenSearch query for the results enclosures

In [26]:
import cioppy

In [27]:
ciop = cioppy.Cioppy()

In [28]:
search = ciop.search(end_point = root.attrib['href'],
                     params = [],
                     output_fields='enclosure')

FileNotFoundError: [Errno 2] No such file or directory: '/usr/bin/opensearch-client': '/usr/bin/opensearch-client'

In [364]:
for result in search:
    print result['enclosure']

http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SMC_A000506_20150728T113618_LULC.zip
http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SMD_A000506_20150728T113618_LULC.zip
http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SNC_A000506_20150728T113618_LULC.zip
http://territorial-planning.deimos.nextgeoss.terradue.com/data/run/workflow/0000010-190709165332500-oozie-oozi-W/_results/L2A_T29SND_A000506_20150728T113618_LULC.zip


### Download an output file

The following section can be leveraged when published results are available.

In [24]:
#search[0]['enclosure']

In [25]:
#r = requests.get(search[0]['enclosure'],stream=True)

In [26]:
#with open('output', 'wb') as fd:
#    for chunk in r.iter_content(chunk_size=128):
#        fd.write(chunk)